In [22]:
import folium
import pandas as pd
import sqlite3
import streamlit as st
from streamlit_folium import folium_static

conn = sqlite3.connect('SNCF_LOST.db')

df_gare = pd.read_sql_query("SELECT * FROM Gare", conn)

annee = st.sidebar.slider("Choisir une année", 2019, 2022, 2022)
type_objet = st.sidebar.selectbox("Choisir un type d'objet", df_objet["TypeObjet"].unique())

df_objet = pd.read_sql_query(f"SELECT GarePerte, COUNT(*) AS NbObjets FROM ObjetPerdu WHERE AnneePerte = '{annee}' AND TypeObjet = '{type_objet}' GROUP BY GarePerte", conn)

df_map = pd.merge(df_gare, df_objet, on="GarePerte")

m = folium.Map(location=[48.8534, 2.3488], zoom_start=12)

for index, row in df_map.iterrows():
    tooltip = f"{row['Nom']}<br>Fréquentation {annee}: {row[f'Freq_{annee}']}<br>Objets perdus: {row['NbObjets']}"
    folium.Marker([row['Latitude'], row['Longitude']], tooltip=tooltip).add_to(m)

st.title("Carte des objets perdus dans les gares")
st.sidebar.title("Paramètres")
st.sidebar.write("Choisissez une année et un type d'objet pour afficher le nombre d'objets perdus dans chaque gare.")
st.sidebar.write("")
st.sidebar.write("")
st.sidebar.write("")
folium_static(m)


AttributeError: partially initialized module 'streamlit' has no attribute 'multiselect' (most likely due to a circular import)

In [ ]:
import sqlite3
import pandas as pd
import folium
from folium.plugins import MarkerCluster, Search, Fullscreen, BeautifyIcon

# Création de la carte centrée sur Paris
paris_map = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

conn = sqlite3.connect('SNCF_LOST.db')


# Chargement des données
temperature_data = pd.read_sql_query("SELECT * FROM Temperature", conn)
objet_perdu_data = pd.read_sql_query("SELECT * FROM ObjetPerdu", conn)
gare_data = pd.read_sql_query("SELECT * FROM Gare", conn)

# Jointure des données sur les gares et les objets perdus
merged_data = pd.merge(objet_perdu_data, gare_data, left_on='GarePerte', right_on='Nom')

# Calcul du nombre d'objets perdus par gare et par année
objects_count = merged_data.groupby(['Nom', 'AnneePerte', 'TypeObjet']).size().reset_index(name='object_count')

# Calcul de la fréquentation de chaque gare pour chaque année
freq_data = gare_data.melt(id_vars=['Nom', 'Latitude', 'Longitude'], var_name='Annee', value_name='Freq')

# Création d'un groupe de marqueurs pour chaque année
for year in objects_count['AnneePerte'].unique():

    year_data = objects_count[objects_count['AnneePerte'] == year]
    
    # Création d'un groupe de marqueurs pour chaque type d'objet
    for obj_type in year_data['TypeObjet'].unique():

        obj_type_data = year_data[year_data['TypeObjet'] == obj_type]
        
        # Création d'un groupe de marqueurs pour chaque gare
        marker_cluster = MarkerCluster(name=f'{year} - {obj_type}').add_to(paris_map)
        
        # Ajout des marqueurs pour chaque gare
        for _, row in obj_type_data.iterrows():
            gare_info = freq_data[(freq_data['Nom'] == row['Nom']) & (freq_data['Annee'] == row['AnneePerte'])]
            freq = gare_info['Freq'].iloc[0]
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                icon=BeautifyIcon(
                    icon='fa-cube',
                    inner_icon_style='color:white;font-size:1.5em;',
                    background_color=obj_type_data['object_count'].apply(lambda x: 'red' if x > 30 else 'orange').tolist(),
                    border_color='transparent'
                ),
                tooltip=f"{row['Nom']} - {obj_type} - {row['object_count']} objects - {freq} passengers",
                popup=f"{row['Nom']} - {obj_type} - {row['object_count']} objects - {freq} passengers",
            ).add_to(marker_cluster)

# Ajout d'un bouton pour rechercher une gare
search = Search(
    layer=marker_cluster,
    geom_type='Marker',
    placeholder='Search for a station',
    collapsed=False,
    search_zoom=14
).add_to(paris_map)

# Ajout d'un bouton pour activer le mode plein écran
Fullscreen(
    position='topright',
    force_separate_button=True
).add_to(paris_map)

# Affichage de la carte
paris_map